<a href="https://colab.research.google.com/github/boshuaiYu/CaiCai_DL/blob/main/CaiCai_DL_week3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 神经网络的损失函数

In [2]:
import torch
from torch.nn import MSELoss

In [7]:
torch.random.manual_seed(420)
yhat = torch.randn((50,),dtype=torch.float32)
y = torch.randn((50,),dtype=torch.float32)

In [8]:
criterion = MSELoss()
loss = criterion(yhat,y)
loss

tensor(2.4869)

<font color = "red">MSELoss()函数中只有一个reduction参数

<font color = "yellow">reduction=mean，输出MSE</font>

<font color = "green">reduction=sum，输出SSE</font>

In [11]:
criterion = MSELoss(reduction="mean")
loss = criterion(yhat,y)
loss

tensor(2.4869)

In [12]:
criterion = MSELoss(reduction="sum")
loss = criterion(yhat,y)
loss

tensor(124.3458)

**tensor实现二分类交叉熵函数**

In [13]:
import torch
import time

In [26]:
N = 3*pow(10,6)
torch.random.manual_seed(420)
X = torch.rand((N,4),dtype=torch.float32)
w = torch.rand((4,1),dtype=torch.float32)
y = torch.randint(low=0,high=2,size=(N,1),dtype=torch.float32)

zhat = torch.mm(X,w)
sigma = torch.sigmoid(zhat)

In [27]:
Loss = -(y*torch.log(sigma)+(1-y)*torch.log(1-sigma))
Loss.shape,Loss.max(axis=0)

(torch.Size([3000000, 1]), torch.return_types.max(
 values=tensor([2.4099]),
 indices=tensor([1657067])))

In [38]:
start = time.time()
Loss = -(1/N)*torch.sum(y*torch.log(sigma)+(1-y)*torch.log(1-sigma)) # 张量运行的速度特别快
end = time.time()
print("Loss is %.8f，time is %.8f s" % (Loss,end-start))

Loss is 0.86853886，time is 0.03792000 s


In [36]:
start = time.time()
Loss = -(1/N)*sum(y*torch.log(sigma)+(1-y)*torch.log(1-sigma))
end = time.time()
print("Loss is %.8f，time is %.8f s" % (Loss,end-start))

Loss is 0.86916202，time is 39.66135931 s


**Pytorch中的类实现二分类交叉熵函数**

**<font color="yellow">方法一： nn提供了两个类：BCEWithLogitsLoss以及BCELoss。虽然PyTorch官方没有直接明确，但实际上两个函数所需要输入的参数不同。</font>**

**<font color="red">BCEWithLogitsLoss内置了sigmoid函数与交叉熵函数，它会自动计算输入值的sigmoid值，因此需要输入zhat与真实标签，且顺序不能变化，zhat必须在前。</font>**

**<font color="red">BCELoss中只有交叉熵函数，没有sigmoid层，因此需要输入sigma与真实标签，且顺序不能变化。</font>**

**<font color="yellow">同时，这两个函数都要求真实值在后面且预测值与真实标签的数据类型以及结构必须相同</font>**

In [39]:
import torch.nn as nn
criterion = nn.BCELoss()
loss = criterion(sigma,y)
loss

tensor(0.8685)

In [40]:
criterion = nn.BCEWithLogitsLoss()
loss = criterion(zhat,y)
loss

tensor(0.8685)

**<font color="yellow">方法二： functional库中的计算函数</font>**


**F.binary_cross_entropy_with_logits**

**F.binary_cross_entropy**

**多分类交叉熵损失函数**

**<font color = "yellow">方法一: 使用LogSoftmax()和NLLLoss()函数**

In [41]:
import torch
import torch.nn as nn

In [44]:
N = 3*pow(10,6)
torch.random.manual_seed(420)
X = torch.rand((N,4),dtype=torch.float32)
w = torch.rand((4,3),dtype=torch.float32)
y = torch.randint(low=0,high=3,size=(N,1),dtype=torch.float32)

zhat = torch.mm(X,w)

In [52]:
logsm = nn.LogSoftmax(dim=1)
logsigma = logsm(zhat)
logsigma

tensor([[-0.4011, -1.7739, -1.8281],
        [-0.6282, -1.4567, -1.4548],
        [-0.9008, -1.1484, -1.2853],
        ...,
        [-0.5346, -1.4548, -1.7111],
        [-0.4832, -1.4721, -1.8725],
        [-0.4189, -1.6581, -1.8855]])

In [48]:
criterion = nn.NLLLoss()
criterion(logsigma,y.flatten().long()) # 调用NLLLoss计算需要输入标签为1维并且为整型 

tensor(1.2305)

**<font color = "yellow">方法2： 直接调用CrossEntropyLoss函数**

In [50]:
criterion = nn.CrossEntropyLoss()
criterion(zhat,y.flatten().long())

tensor(1.2305)